# Python for Statistical Analyis

In [ ]:
import pandas as pd
import seaborn as sns
import hvplot.pandas
from matplotlib import pyplot as plt

## The Iris Dataset

In [ ]:
data = sns.load_dataset('iris')

In [ ]:
data

In [ ]:
type(data)

## Exploratory Analysis

In [ ]:
data.head()

In [ ]:
data['species'].unique()

### Summary Statistics

In [ ]:
data.describe()

### Summary Statistics By Group

In [ ]:
data.groupby('species').count()

In [ ]:
data.groupby('species').mean()

In [ ]:
data.groupby('species').min()

In [ ]:
data.groupby('species').max()

## Data Visualization

### Scatter Matrices

In [ ]:
pd.plotting.scatter_matrix(data, figsize=(10, 10))
plt.show()

In [ ]:
sns.pairplot(data, hue='species')
plt.show()

In [ ]:
hvplot.scatter_matrix(data, c='species', diagonal='kde')

### Grouped Box Plots

In [ ]:
data_long = pd.melt(data, id_vars='species')
data_long

In [ ]:
(data_long.groupby('variable')
          .boxplot(by='species', sharey=False, figsize=(10, 10)))
plt.show()

In [ ]:
sns.catplot(data=data_long, x='species', y='value', sharey=False,
            col='variable', col_wrap=2, kind='box')
plt.show()

In [ ]:
data_long.hvplot.box(y='value', by='species', groupby='variable', height=400)

## ANOVA

### ANOVA using SciPy

In [ ]:
import scipy.stats as stats

In [ ]:
grouped = data.groupby('species')['sepal_width']

In [ ]:
rows = []
for name, values in grouped:
    statistic, pvalue = stats.shapiro(values)
    rows.append({'species': name, 'statistic': statistic, 'pvalue': pvalue})
pd.DataFrame(rows).set_index('species')

In [ ]:
for name, values in grouped:
    stats.probplot(values, plot=plt)
    plt.title(name)
    plt.show()

In [ ]:
stats.bartlett(grouped.get_group('setosa'), 
               grouped.get_group('versicolor'),
               grouped.get_group('virginica'))

In [ ]:
stats.f_oneway(grouped.get_group('setosa'), 
               grouped.get_group('versicolor'),
               grouped.get_group('virginica'))

### ANOVA using Statsmodels

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import statsmodels.stats.multicomp as mc

In [ ]:
model = ols("sepal_width ~ C(species)", data=data).fit()
model.summary()

In [ ]:
sm.qqplot(model.resid)
plt.show()

In [ ]:
anova = anova_lm(model)
anova

In [ ]:
comp = mc.MultiComparison(data['sepal_width'], data['species'])
post_hoc_res = comp.tukeyhsd()
post_hoc_res.summary()

### ANOVA using Pingouin

In [ ]:
import pingouin as pg

In [ ]:
for name, values in data.groupby('species')['sepal_width']:
    pg.qqplot(values)
    plt.title(name)
    plt.show()

In [ ]:
pg.normality(data, dv='sepal_width', group='species')

In [ ]:
pg.homoscedasticity(data, dv='sepal_width', group='species')

In [ ]:
pg.anova(data, dv='sepal_width', between='species', detailed=True)